In [1]:
import fs from 'fs'
import xlsx from 'node-xlsx';
import { format } from 'date-fns';

In [214]:
// For EMCR
const emcrNumHeaderRows = 4;
const emcrNumEndRows = 0;

const emcrWorksheet = xlsx.parse(fs.readFileSync(`teamsmembers.xlsx`));
const worksheetArrays = emcrWorksheet.find(page => page.name === 'Sheet1')?.data || [];
const emcrMembers = worksheetArrays.slice(emcrNumHeaderRows, worksheetArrays.length-emcrNumEndRows);

In [32]:
const emcrEmails = emcrMembers.map(member => member[7]?.trim().toLowerCase());

In [551]:
// For BCWS
const numHeaderRows = 4;
const numEndRows = 0;

const worksheet = xlsx.parse(fs.readFileSync(`bcwsmembers.xlsx`));
const worksheetArrays = worksheet.find(page => page.name === 'RECOMMIT')?.data || [];
const members = worksheetArrays.slice(numHeaderRows, worksheetArrays.length-numEndRows);

In [552]:
// Find email delta between datasets
const bcwsEmailsInBoth = members.filter(member => member[0]?.toLowerCase() === 'both').map(member => member[3]?.trim().toLowerCase());
const sameEmails = [];
const diffEmails = [];
bcwsEmailsInBoth.forEach(email => {
    if (!emcrEmails.includes(email)) {
        diffEmails.push(email);
    } else {
        sameEmails.push(email);
    }
});

In [221]:
enum UnionMembership {
  BCGEU = 'BCGEU',
  EXCLUDED = 'EXCLUDED',
  BCNU = 'BCNU',
  PEA = 'PEA',
}

enum Status {
  ACTIVE = 'ACTIVE',
  INACTIVE = 'INACTIVE',
  PENDING = 'PENDING',
}

enum ToolsProficiency {
  BASIC = 'BASIC',
  INTERMEDIATE = 'INTERMEDIATE',
  ADVANCED = 'ADVANCED',
}

enum ExperienceLevel {
  PREVIOUSLY_DEPLOYED = 'PREVIOUSLY_DEPLOYED',
  INTERESTED = 'INTERESTED',
}

enum LanguageProficiency {
  BASIC = 'BASIC',
  INTERMEDIATE = 'INTERMEDIATE',
  FLUENT = 'FLUENT',
}

enum LanguageLevelType {
  VERBAL = 'VERBAL',
  WRITTEN = 'WRITTEN',
  BOTH = 'BOTH',
}


interface Personnel {
    firstName: string;
    lastName: string;
    primaryPhone: string;
    secondaryPhone?: string;
    workPhone?: string;
    email: string;
    supervisorFirstName: string;
    supervisorLastName: string;
    supervisorEmail: string;
    unionMembership: UnionMembership;
    remoteOnly: boolean;
    willingToTravel: boolean;
    coordinatorNotes?: string;
    driverLicense?: string[];
    jobTitle?: string;
    homeLocation: string;
    workLocation?: string;
}

interface BcwsPersonnel {
    status: Status;
    employeeId: string;
    dateApplied: Date; // Is this important to Emily? Not in initial dataset
    dateApproved: Date; // Is this important to Emily? Not in initial dataset
    approvedBySupervisor: boolean;
    purchaseCardHolder: boolean;
    division: string;
    paylistId: string;
    liaisonFirstName?: string;
    liaisonLastName?: string;
    liaisonPhoneNumber?: string;
    liaisonEmail?: string;
    emergencyContactFirstName?: string;
    emergencyContactLastName?: string;
    emergencyContactPhoneNumber?: string;
    willingnessStatement: boolean;
    parQ: boolean;
    respectfulWorkplacePolicy: boolean;
    orientation: boolean;
    firstChoiceSection: string;
    secondChoiceSection: string;
    email: string;
}

interface BcwsPersonnelTools {
    tools: {
        tool: number;
        proficiencyLevel: ToolsProficiency;
    }
    email: string;
}

interface BcwsPersonnelRoles {
    roles: {
        role: number;
        expLevel: ExperienceLevel;
    }[];
    email: string;
}

interface BcwsPersonnelLanguages {
    languages: {
        language: string;
        level: LanguageProficiency;
        type: LanguageLevelType;
    }[];
    email: string;
}

interface BcwsPersonnelCerts {
    certs: {
        cert: number;
        expiry?: Date;
    }
    email: string;
}

In [230]:
const header = worksheetArrays[3];
header.map((h, i) => console.log(i, h));
console.log(header[139]);

0 TEAMS Program
1 First Name 
2 Last Name 
3 Email 
4 Employee ID
5 IDIR
6 Ministry 
7 Division
8 Branch
9 Paylist 
10 Region
11 Geographic Location
12 Bargaining Unit
13 PrimaryPhoneNumber
14 SecondaryPhoneNumber
15 Supervisor First Name
16 Supervisor Last Name
17 Supervisor Email
18 Supervisor Phone Number
19 Supervisor Approval Provided
20 Fire Centre
21 1st Choice
22 2nd Choice
23 Demobilization Unit Leader
24 Documentation Unit Leader
25 Fire Behaviour Specialist
26 GIS Specialist - Exam Required
27 Planning Assistant
28 Plans Officer
29 Plans Section Chief
30 Rehab Specialist
31 Resource Unit Leader
32 Situation Unit Leader
33 Demobilization Unit Leader
34 Documentation Unit Leader
35 Fire Behaviour Specialist
36 GIS Specialist - Exam Required
37 Planning Assistant
38 Plans Officer
39 Plans Section Chief
40 Rehab Specialist
41 Resource Unit Leader
42 Situation Unit Leader
43 Asset Management
44 Camp Manager
45 Communication Unit Leader
46 Contract Monitor
47 Dispatcher
48 Facilit

In [223]:
const previousToInterest = {
    23: 33, // Planning
    24: 34,
    25: 35,
    26: 36,
    27: 37,
    28: 38,
    29: 39,
    30: 40,
    31: 41,
    32: 42,
    43: 59, // Log
    44: 60,
    45: 61,
    46: 62,
    47: 63,
    48: 64,
    49: 65,
    50: 66,
    51: 67,
    52: 68,
    53: 69,
    54: 70,
    55: 71,
    56: 72,
    57: 73,
    58: 74,
    75: 86, // Fin
    76: 87,
    77: 88,
    78: 89,
    79: 90,
    80: 91,
    81: 92,
    82: 93,
    83: 94,
    84: 95,
    85: 96,
    97: 103, // Ops
    98: 104,
    99: 105,
    100: 106,
    101: 107,
    102: 108,
    109: 113, // Command
    110: 114,
    111: 115,
    112: 116,
    117: 120, // Aviation
    118: 121,
    119: 122,
}

const getRoleId = (index: number) => {
    switch (index) {
        case 23: case 33: return 1; // DEMOBILIZATION_UNIT_LEADER
        case 24: case 34: return 2; // DOCUMENTATION_UNIT_LEADER
        case 25: case 35: return 3; // FIRE_BEHAVIOUR_SPECIALIST
        case 26: case 36: return 4; // GIS_SPECIALIST
        case 27: case 37: return 5; // PLANNING_ASSISTANT
        case 28: case 38: return 7; // PLANS_OFFICER
        case 29: case 39: return 8; // PLANS_SECTION_CHIEF
        case 30: case 40: return 9; // REHAB_SPECIALIST
        case 31: case 41: return 10; // RESOURCE_UNIT_LEADER
        case 32: case 42: return 11; // SITUATION_UNIT_LEADER
            
        case 43: case 59: return 12; // ASSET_MANAGEMENT
        case 44: case 60: return 13; // CAMP_MANAGER
        case 45: case 61: return 14; // COMMUNICATION_UNIT_LEADER
        case 46: case 62: return 15; // CONTRACT_MONITOR
        case 47: case 63: return 16; // DISPATCHER
        case 48: case 64: return 17; // FACILITIES_UNIT_LEADER
        case 49: case 65: return 18; // FOOD_UNIT_LEADER
        case 50: case 66: return 19; // GROUND_SUPPORT_UNIT_LEADER
        case 51: case 67: return 20; // LOGISTICS_ASSISTANT
        case 52: case 68: return 21; // LOGISTICS_OFFICER
        case 53: case 69: return 22; // LOGISTICS_SECTION_CHIEF
        case 54: case 70: return 23; // MEDICAL_UNIT_LEADER
        case 55: case 71: return 24; // SERVICE_BRANCH_DIRECTOR
        case 56: case 72: return 25; // SUPPLY_UNIT_LEADER
        case 57: case 73: return 26; // SUPPORT_BRANCH_DIRECTOR
        case 58: case 74: return 27; // WAREHOUSE_MANAGER
            
        case 75: case 86: return 28; // ACCOUNTS_PAYABLE
        case 76: case 87: return 29; // COMPENSATION_CLAIMS_UNIT_LEADER
        case 77: case 88: return 30; // CONTRACT_ADMINISTRATION
        case 78: case 89: return 31; // COST_UNIT_LEADER
        case 79: case 90: return 32; // FINANCE_ASSISTANT
        case 80: case 91: return 33; // FINANCE_LIAISON
        case 81: case 92: return 34; // FINANCE_OFFICER
        case 82: case 93: return 35; // FINANCE_SECTION_CHIEF
        case 83: case 94: return 36; // PROCUREMENT_UNIT_LEADER
        case 84: case 95: return 37; // TIME_RECORDER
        case 85: case 96: return 38; // TIME_UNIT_LEADER

        case 97: case 103: return 39; // DIVISION_SUPERVISOR
        case 98: case 104: return 40; // EQUIPMENT_BRANCH_DIRECTOR
        case 99: case 105: return 41; // EQUIPMENT_GROUP_SUPERVISOR
        case 100: case 106: return 42; // LINE_LOCATOR
        case 101: case 107: return 43; // STRIKE_TEAM_LEADER
        case 102: case 108: return 44; // TASK_FORCE_LEADER

        case 109: case 113: return 45; // INFORMATION_ASSISTANT
        case 110: case 114: return 46; // INFORMATION_OFFICER
        case 111: case 115: return 47; // LIAISON_OFFICER
        case 112: case 116: return 48; // SAFETY_OFFICER
        
        case 117: case 120: return 49; // AVIATION_ASSISTANT
        case 118: case 121: return 50; // HELI_COORDINATOR
        case 119: case 122: return 51; // HELIBASE_MANAGER
    }
}

const roleIdProficiency = (index: number) => {
    return { role: getRoleId(index), expLevel: previousToInterest[index] ? 'PREVIOUSLY_DEPLOYED' : 'INTERESTED' }
}

const getRoles = (rawRoles: (string | undefined)[]) => {
    const roles = [];
    rawRoles.forEach((r, i) => {
        if (r?.trim().toLowerCase() === 'x') {
            const proficiency = roleIdProficiency(i+23);
            if (!roles.find((r) => r.role === proficiency.role)) {
                roles.push(proficiency);
            }
        }
    });
    return roles;
}

In [224]:
const getUnionMembership = (unionValue?: string | undefined | number): UnionMembership => {
    let value = typeof unionValue === 'number' ? unionValue.toString() : unionValue;
    const uv = value?.trim().toLowerCase();
    if (uv === 'pea') {
        return UnionMembership.PEA;
    } else if (uv?.includes('excluded') || !unionValue || unionValue === 'n/a') {
        return UnionMembership.EXCLUDED;
    } else {
        return UnionMembership.BCGEU;
    }
}

In [460]:
const locationIds = {
    '100 mile house': 1,
    '150 mile house': 2,
    'abbotsford': 3,
    'brentwood bay': 4,
    'burnaby': 5,
    'burns lake': 6,
    'bonnington falls': 7,
    'campbell river': 8,
    'castlegar': 9,
    'cumberland': 10,
    'coquitlam': 11,
    'courtenay': 12,
    'cranbrook': 13,
    'dawson creek': 14,
    'duncan': 15,
    'elkford': 16,
    'enderby': 17,
    'esquimalt': 18,
    'fort st. john': 19,
    'fort nelson': 20,
    'kamloops': 21,
    'kelowna': 22,
    'kimberley': 23,
    'langford': 24,
    'langley': 25,
    'lillooet': 26,
    'mackenzie': 27,
    'maple ridge': 28,
    'merritt': 29,
    'mill bay': 30,
    'mission': 31,
    'nanaimo': 32,
    'nelson': 33,
    'new westminster': 34,
    'north vancouver': 35,
    'parksville': 36,
    'penticton': 37,
    'port alberni': 38,
    'port mcneil': 39,
    'prince george': 40,
    'qualicum beach': 41,
    'quesnel': 42,
    'revelstoke': 43,
    'richmond': 44,
    'saanich': 45,
    'saanichton': 46,
    'salmon arm': 47,
    'sechelt': 48,
    'sidney': 49,
    'smithers': 50,
    'sorrento': 51,
    'surrey': 52,
    'terrace': 53,
    'ucluelet': 54,
    'vancouver': 55,
    'vernon': 56,
    'victoria': 57,
    'whistler': 58,
    'williams lake': 59,
    'vanderhoof': 60,
    'sooke': 61,
    'alexis creek': 62,
    'bella coola': 63,
    'boston bar': 64,
    'chilliwack': 65,
    'cultus lake': 66,
    'masset': 67,
    'pemberton': 68,
    'powell river': 69,
    'squamish': 70,
    'clearwater': 71,
    'lytton': 72,
    'princeton': 73,
    'atlin': 74,
    'dease lake': 75,
    'hazelton': 76,
    'houston': 77,
    'prince rupert': 78,
    'blue river': 79,
    'chetwynd': 80,
    'fort st. james': 81,
    'valemount': 82,
    'golden': 83,
    'grand forks': 84,
    'invermere': 85,
    'nakusp': 86
}

In [292]:
const divisionIds = {
  "office of the chief forester-for": 1,
  "integrated resource operations division-for": 2,
  "timber range and economics-for": 3,
  "forest resiliency and archaeology-for": 4,
  "north area-for": 5,
  "south area-for": 6,
  "coast area-for": 7,
  "bc timber sales-for": 8,
  "bc wildfire-for": 9,
  "land use planning and cumulative effects-wlrs": 10,
  "land use planning and cumulative effects division-wlrs": 10,
  "resource stewardship-wlrs": 11,
  "resource stewardship-division-wlrs": 11,
  "water fisheries and coast-wlrs": 12,
  "water fisheries and coast division-wlrs": 12,
  "reconciliation lands and natural resource policy-wlrs": 13,
  "reconciliation, lands and natural resource policy division-wlrs": 13,
  "permitting transformation-wlrs": 14,
  "natural resource information and digital services-wlrs": 15,
  "corporate services for natural resource ministries-wlrs": 16,
  "corporate services for the natural resource ministries-wlrs": 16,
  "conservation and recreation division-env": 17,
  "recreation strategy and service transformation-env": 18,
  "climate action secretariat-env": 19,
  "environmental protection division-env": 20,
  "strategic services division-env": 21,
  "deputy minister's office-af": 22,
  "agriculture resource division-af": 23,
  "science policy and inspection division-af": 24,
  "climate resilience competitiveness and reconciliation division-af": 25,
  "corporate management services-ag": 26,
  "information systems-ag": 27,
  "bc prosecution service-ag": 28,
  "court services branch-ag": 29,
  "independent investigations office-ag": 30,
  "investigation and standards office-ag": 31,
  "justice services branch-ag": 32,
  "legal services branch-ag": 33,
  "multiculturalism and anti-racism branch-ag": 34,
  "bc corrections-pssg": 35,
  "community corrections-pssg": 35,
  "bc coroners service-pssg": 36,
  "cannabis consumer protection and corporate policy branch-pssg": 37,
  "community safety and crime prevention-pssg": 38,
  "gaming policy and enforcement branch-pssg": 39,
  "liquor and cannabis regulation branch-pssg": 40,
  "office of the fire commissioner-pssg": 41,
  "policing and security-pssg": 42,
  "roadsafetybc-pssg": 43,
  "deputy minister's office-mcf": 44,
  "information services-mcf": 45,
  "service delivery division-mcf": 46,
  "strategic integration policy & legislation division-mcf": 47,
  "partnership & indigenous engagement-mcf": 48,
  "finance & corporate services-mcf": 49,
  "strategic services division-mcf": 50,
  "office of the provincial director & aboriginal services-mcf": 51,
  "director & aboriginal services-for": 51,
  "deputy minister's office-citz": 52,
  "bc data service-citz": 53,
  "connectivity-citz": 54,
  "corporate services-citz": 55,
  "deal management office-citz": 56,
  "government digital experience-citz": 57,
  "ocio - government chief information officer-citz": 58,
  "ocio - corporate information and records management office-citz": 59,
  "corporate information and records management office-citz": 59,
  "ocio - enterprise services-citz": 60,
  "procurement and supply-citz": 61,
  "real property-citz": 62,
  "real property division-citz": 62,
  "service bc-citz": 63,
  "deputy minister's office-ecc": 64,
  "child care-ecc": 65,
  "education programs-ecc": 66,
  "governance and analytics-ecc": 67,
  "learning-ecc": 68,
  "resource management-ecc": 69,
  "services & technology-ecc": 70,
  "dmo and associate dmo-emcr": 71,
  "corporate services-emcr": 72,
  "disaster recovery-emcr": 73,
  "disaster risk management-emcr": 74,
  "partnerships engagement and legislation-emcr": 75,
  "regional operations-emcr": 76,
  "deputy minister's office-emli": 77,
  "electricity and utility regulation division-emli": 78,
  "energy decarbonization division-emli": 79,
  "energy resources division-emli": 80,
  "responsible mining and competitiveness division-emli": 81,
  "mines health safety & enforcement division-emli": 82,
  "strategic and indigenous partnerships division-emli": 83,
  "corporate services-fin": 84,
  "gender equity office-fin": 85,
  "office of the comptroller general-fin": 86,
  "policy & legislation division-fin": 87,
  "provincial treasury-fin": 88,
  "revenue division and anti-money laundering secretariat-fin": 89,
  "treasury board staff-fin": 90,
  "deputy minister's office-fin": 91,
  "deputy minister's office (dmo)-hlth": 92,
  "associate deputy minister's offices-hlth": 93,
  "finance and corporate services-hlth": 94,
  "health sector information analysis and reporting-hlth": 95,
  "health sector information management / information technology-hlth": 96,
  "health sector workforce and beneficiary services-hlth": 97,
  "hospital and provincial health services-hlth": 98,
  "mental health and substance use-hlth": 99,
  "office of the provincial health officer-hlth": 100,
  "office of the seniors advocate-hlth": 101,
  "pharmaceutical laboratory and blood services-hlth": 102,
  "population and public health-hlth": 103,
  "primary care-hlth": 104,
  "seniors' services-hlth": 105,
  "strategic innovation-hlth": 106,
  "strategy management and people office-hlth": 107,
  "deputy minister's office-hous": 108,
  "housing and land use policy-hous": 109,
  "homelessness partnerships and housing supports-hous": 110,
  "strategy governance and accountability-hous": 111,
  "housing innovations-hous": 112,
  "deputy minister's office-irr": 113,
  "negotiations & regional operations division-irr": 114,
  "negotiations and regional operations-irr": 114,
  "negotiations and regional operations division-irr": 114,
  "reconciliation transformation & strategies division-irr": 115,
  "strategic partnerships & initiatives division-irr": 116,
  "sustainable economy-jedi": 117,
  "investment division-jedi": 118,
  "small business and economic development-jedi": 119,
  "trade and industry development division-jedi": 120,
  "employment standards-lbr": 121,
  "policy & legislation-lbr": 122,
  "labour division-lbr": 123,
  "deputy minister's office-mmha": 124,
  "corporate services-mmha": 125,
  "child youth & mental health policy-mmha": 126,
  "provincial support office-mmha": 127,
  "substance use policy-mmha": 128,
  "treatment & recovery-mmha": 129,
  "deputy minister's office-muni": 130,
  "immigration services and strategic planning-muni": 131,
  "efo management services-muni": 132,
  "local government-muni": 133,
  "deputy minister's office-psfs": 134,
  "finance technology & management services-psfs": 135,
  "post-secondary policy & programs-psfs": 136,
  "labour market development-psfs": 137,
  "governance legislation & engagement-psfs": 138,
  "employment and labour market services division-sdpr": 139,
  "employment & labour market services-sdpr": 139,
  "employment & labour market services division-sdpr": 139,
  "corporate services-sdpr": 140,
  "research innovation and policy-sdpr": 141,
  "service delivery-sdpr": 142,
  "service delivery division-sdpr": 142,
  "accessibility directorate-sdpr": 143,
  "tourism sector strategy-tacs": 144,
  "arts and culture-tacs": 145,
  "sport and creative-tacs": 146,
  "management services division-tacs": 147,
  "deputy minister's office-moti": 148,
  "associate deputy ministers-moti": 149,
  "highways and regional services division-moti": 150,
  "highways and regional services-moti": 150,
  "highways & regional services division-moti": 150,
  "highways and regional services division: south coast region-moti": 151,
  "highways and regional services division: southern interior region-moti": 152,
  "highways and regional services division: northern region-moti": 153,
  "policy programs and partnerships division-moti": 154,
  "integrated transportation and infrastructure services division-moti": 155,
  "strategic and corporate priorities division-moti": 156,
  "finance and risk management division-moti": 157
}

In [574]:
const ministryToEnum = {
    "Ministry of Water, Land and Resource Stewardship": "wlrs",
    "Ministry of Forests": "for",
    "Ministry of Transportation and Infrastructure": "moti",
    "Ministry of Environment and Climate Change Strategy": "env",
    "Ministry of Children and Family Development": "mcf",
    "Public Safety & Solicitor General": "pssg",
    "Ministry of Social Development and Poverty Reduction": "sdpr",
    "Ministry of Citizens' Services": "citz",
    "Ministry of Labour": "lbr",
    "Attorney General": "ag",
    "Ministry of Municipal Affairs": "muni",
    "Ministry of Energy, Mines and Low Carbon Innovation": "emli",
    "Ministry of Education and Child Care": "ecc",
    "Tourism, Arts, Culture and Sport": "tacs",
    "Ministry of Indigenous Relations and Reconciliation": "irr",
    "Ministry of Agriculture and Food": "af",
    "Ministry of Mental Health and Addictions": "mmha",
    "Ministry of Finance": "fin",
    "Ministry of Health": "hlth",
    "Public Service Agency": "psa"
}

In [479]:
const sectionToEnum = {
    'logistics': 'LOGISTICS',
    'command': 'COMMAND',
    'comand': 'COMMAND',
    'financeAdmin': 'FINANCE_ADMIN',
    'planning': 'PLANNING',
    'operations': 'OPERATIONS',
    'aviation': 'AVIATION',
}

In [480]:
const getStatus = (member: any) => {
    if (member[0] === 'PENDING') {
        return 'PENDING';
    }
    else if (
        member[19]?.toLowerCase() === 'yes' &&
        member[158]?.toLowerCase() === 'x' &&
        member[159]?.toLowerCase() === 'x' &&
        member[160]?.toLowerCase() === 'x' &&
        member[161]?.toLowerCase() === 'x'
    ) {
        return 'ACTIVE';
    }
    return 'INACTIVE';
}

In [556]:
const incomingMembers = members.filter(m => !!m[0]);

In [644]:
// const ministries = incomingMembers.map(m => m[6]?.trim());
// ministries.forEach((m, i) => console.log(ministryToEnum[m] || m, i));
const divisions = incomingMembers.map(m => `${m[7]?.trim()?.toLowerCase()}-${ministryToEnum[m[6]?.trim()]}`);

In [643]:
const emcrPersonnel = {
  "email@gov.bc.ca"	:	"uuid-in-our-system",
  }

In [553]:
const existingIds = personnel.filter((p => !!emcrPersonnel[p.email])).reduce((acc, p) => {
    acc[p.email] = emcrPersonnel[p.email];
    return acc;
}, {});

In [580]:
const personnel: any[] = incomingMembers.map(member => {
    return {
        firstName: member[1]?.trim(),
        lastName: member[2]?.trim(),
        primaryPhone: member[13]?.replace(/(?<!^)\+|[^\d+]+/g, ''),
        secondaryPhone: member[14]?.replace(/(?<!^)\+|[^\d+]+/g, ''),
        email: member[3]?.trim(),
        supervisorFirstName: member[15]?.trim(),
        supervisorLastName: member[16]?.trim(),
        supervisorEmail: member[17]?.trim(),
        unionMembership: getUnionMembership(member[12]),
        remoteOnly: false,
        willingToTravel: true,
        driverLicense: ['CLASS_5'],
        homeLocation: locationIds[member[11]?.trim()?.toLowerCase()],
        ministry: ministryToEnum[member[6]?.trim()],
        division: member[7]?.trim(),
    }
});

const bcwsPersonnel: any[] = incomingMembers.map(member => {
    const status = getStatus(member);
    const divisionMinistry = `${member[7]?.trim()?.toLowerCase()}-${ministryToEnum[member[6]?.trim()]}`;
    const divisionId = divisionIds[divisionMinistry];
    let coordinatorNotes = 'Please follow up with this member to confirm: Willingness to Travel, Remote Only, Proficiency with Tools and Languages, Certificate Expirations';
    if (['048-0000', '133-0000', '128-0000', '112-104'].includes(member[9]) || !member[9]) { coordinatorNotes += '. Please also confirm the PaylistId' };
    return {
        email: member[3],
        status: status,
        employeeId: (member[4].toString()).padStart(6, '0'),
        dateApplied: '2024-05-01 12:00:00',
        dateApproved: status === 'ACTIVE' ? '2024-05-03 12:00:00' : null,
        approvedBySupervisor: member[19]?.toLowerCase() === 'yes',
        purchaseCardHolder: member[133]?.toLowerCase() === 'x',
        paylistId: member[9],
        willingnessStatement: member[159]?.toLowerCase() === 'x',
        parQ: member[158]?.toLowerCase() === 'x',
        respectfulWorkplacePolicy: member[157]?.toLowerCase() === 'x',
        orientation: status === 'ACTIVE' ? true : false,
        firstChoiceSection: sectionToEnum[member[21]?.trim()],
        secondChoiceSection: sectionToEnum[member[22]?.trim()] || undefined,
        coordinatorNotes,
    }
});

const roles: any[] = incomingMembers.map(member => {
    const roles = getRoles(member.slice(23, 123));
    return {
        roles,
        email: member[3],
    }
});

const languages: any[] = incomingMembers.map(member => {
    const languages = [];
    if (member[123]?.trim().toLowerCase() === 'x') {
        languages.push({ language: 'Spanish', level: 'BASIC', type: 'BOTH' });
    }
    if (member[130]?.trim().toLowerCase() === 'x') {
        languages.push({ language: 'French', level: 'BASIC', type: 'BOTH' });
    }
    return {
        languages,
        email: member[3],
    };
});

const tools: any[] = incomingMembers.map(member => {
    const tools = [];
    if (member[125]?.trim().toLowerCase() === 'x') { // CAS
        tools.push({ tool: 2, proficiencyLevel: 'BASIC' });
    }
    if (member[126]?.trim().toLowerCase() === 'x') { // DEC
        tools.push({ tool: 3, proficiencyLevel: 'BASIC' });
    }
    if (member[127]?.trim().toLowerCase() === 'x') { // Excel
        tools.push({ tool: 4, proficiencyLevel: 'BASIC' });
    }
    if (member[128]?.trim().toLowerCase() === 'x') { // FACE
        tools.push({ tool: 5, proficiencyLevel: 'BASIC' });
    }
    if (member[136]?.trim().toLowerCase() === 'x') { // RRT
        tools.push({ tool: 7, proficiencyLevel: 'BASIC' });
    }
    if (member[138]?.trim().toLowerCase() === 'x') { // Visio
        tools.push({ tool: 8, proficiencyLevel: 'BASIC' });
    }
    return {
        tools,
        email: member[3],
    }
});

const certs: any[] = incomingMembers.map(member => {
    const certs = [];
    if (member[129]?.trim().toLowerCase() === 'x') { // Forklift
        certs.push({ cert: 1 });
    }
    if (member[135]?.trim().toLowerCase() === 'x') { // Radio Operator
        certs.push({ cert: 3 });
    }
    if (member[139]?.trim().toLowerCase() === 'x') { // Volunteer Fire Dept
        certs.push({ cert: 4 });
    }
    if (member[134]?.trim().toLowerCase() === 'x') { // Quad / ATV
        certs.push({ cert: 5 });
    }
    if (member[131]?.trim().toLowerCase() === 'x') { // OFA I
        certs.push({ cert: 8 });
    }
    if (member[132]?.trim().toLowerCase() === 'x') { // OFA II
        certs.push({ cert: 9 });
    }
    return {
        certs,
        email: member[3],
    }
});

In [581]:
const noDivisionPushbacks = bcwsPersonnel.filter(p => p.division === 0 || !p.email);
const migrateWithDivision = bcwsPersonnel.filter(p => p.division !== 0 && !!p.email);

const preexistingMembers = migrateWithDivision.filter(p => emcrPersonnel[p.email.toLowerCase()]);
const netNewBcwsMembers = migrateWithDivision.filter(p => !emcrPersonnel[p.email.toLowerCase()]);

const netNewEmails = netNewBcwsMembers.reduce((acc, mem) => {
    acc[mem.email.toLowerCase()] = 1;
    return acc;
}, {});

const netNewPersonnel = personnel.filter(p => !!p.email && !!netNewEmails[p.email.toLowerCase()]);

In [559]:
const pushbackEmailList = {
  'email@gov.bc.ca': 1,
}

In [642]:
const pushbackPersonnel = bcwsPersonnel.filter(p => pushbackEmailList[p.email.toLowerCase()]);
const preexistingPushbacks = pushbackPersonnel.filter(p => emcrPersonnel[p.email.toLowerCase()]);
const netNewBcwsPushbacks = pushbackPersonnel.filter(p => !emcrPersonnel[p.email.toLowerCase()]);

const netNewPushbackEmails = netNewBcwsPushbacks.reduce((acc, mem) => {
    acc[mem.email.toLowerCase()] = 1;
    return acc;
}, {});

const netNewPushbackPersonnel = personnel.filter(p => !!p.email && !!netNewPushbackEmails[p.email.toLowerCase()]);

In [631]:
const sqlPath = 'import-bcws.sql';
await fs.truncate(sqlPath, () => { console.log('deleted file') });

deleted file


In [624]:
// Net New Members
for (const p of netNewPushbackPersonnel) {
    await fs.appendFileSync(sqlPath,
        `\nINSERT INTO personnel ("first_name", "last_name", "primary_phone", "secondary_phone", "email", "supervisor_first_name", "supervisor_last_name", "supervisor_email", "union_membership", "remote_only", "willing_to_travel", "driver_licenses", "home_location", "ministry", "division")
        VALUES (
            '${p.firstName}', '${p.lastName}', ${p.primaryPhone || null}, ${p.secondaryPhone || null}, '${p.email}', '${p.supervisorFirstName}', '${p.supervisorLastName}', '${p.supervisorEmail}', '${p.unionMembership}', false, true, '${p.driverLicense}', ${p.homeLocation}, '${p.ministry.toUpperCase()}', '${p.division.replace("'", "''")}'
        );`);
}

await fs.appendFileSync(sqlPath, `\n\n\n`);

In [626]:
// Net new bcws
for (const p of netNewBcwsPushbacks) {
    await fs.appendFileSync(sqlPath,
        `\nINSERT INTO bcws_personnel ("personnel_id", "status", "employee_id", "date_applied", "date_approved", "approved_by_supervisor", "purchase_card_holder", "paylist_id", "coordinator_notes", "willingess_statement", "par_q", "workplace_policy", "orientation", "first_choice_section", "second_choice_section")
        VALUES (
            (SELECT id FROM personnel WHERE email = '${p.email}'),
            '${p.status}', '${p.employeeId}', '${p.dateApplied}', ${p.dateApproved ? "'" + p.dateApproved + "'" : null}, ${p.approvedBySupervisor}, ${p.purchaseCardHolder}, '${p.paylistId}', '${p.coordinatorNotes}', ${p.willingnessStatement}, ${p.parQ}, ${p.respectfulWorkplacePolicy}, ${p.orientation}, '${p.firstChoiceSection}', ${p.secondChoiceSection ? "'" + p.secondChoiceSection + "'" : null}
        );`);
}
await fs.appendFileSync(sqlPath, `\n\n\n`);

In [628]:
// Crossover BCWS Personnel
for (const p of preexistingPushbacks) {
    await fs.appendFileSync(sqlPath,
        `\nINSERT INTO bcws_personnel ("personnel_id", "status", "employee_id", "date_applied", "date_approved", "approved_by_supervisor", "purchase_card_holder", "paylist_id", "coordinator_notes", "willingess_statement", "par_q", "workplace_policy", "orientation", "first_choice_section", "second_choice_section")
        VALUES (
            '${emcrPersonnel[p.email.toLowerCase()]}', '${p.status}', '${p.employeeId}', '${p.dateApplied}', ${p.dateApproved ? "'" + p.dateApproved + "'" : null}, ${p.approvedBySupervisor}, ${p.purchaseCardHolder}, '${p.paylistId}', '${p.coordinatorNotes}', ${p.willingnessStatement}, ${p.parQ}, ${p.respectfulWorkplacePolicy}, ${p.orientation}, '${p.firstChoiceSection}', ${p.secondChoiceSection ? "'" + p.secondChoiceSection + "'" : null}
        );`);
}
await fs.appendFileSync(sqlPath, `\n\n\n`);

In [630]:
// new roles
for (const r of roles) {
    if (r.roles.length === 0) { continue; }
    if (!pushbackEmailList[r.email?.toLowerCase()]) { continue; }
    const selectRoles = r.roles.reduce((acc, role) => {
        acc.push(`SELECT id AS user_id, ${role.role} AS role_id, '${role.expLevel}'::"role-experience-level" AS exp_level FROM PersonnelId`);
        return acc;
    }, []);
    await fs.appendFileSync(sqlPath,
        `\nWITH PersonnelId AS (SELECT id FROM personnel WHERE LOWER(email) = '${r.email.toLowerCase()}')
        INSERT INTO bcws_personnel_roles ("personnel_id", "role_id", "exp_level")
        SELECT "user_id", "role_id", "exp_level"
        FROM (
            ${selectRoles.join('\nUNION ALL\n')}
        ) as subquery;\n`);
}
await fs.appendFileSync(sqlPath, `\n\n\n`);


In [632]:
// Net new Languages
for (const l of languages) {
    if (l.languages.length === 0) { continue; }
    if (!pushbackEmailList[l.email?.toLowerCase()]) { continue; }
    const selectLanguages = l.languages.reduce((acc, language) => {
        acc.push(`SELECT id AS user_id, '${language.language}' AS language, '${language.level}'::"language-proficiency" AS level, '${language.type}'::"language-fluency-type" AS level_type FROM PersonnelId`);
        return acc;
    }, []);
    await fs.appendFileSync(sqlPath,
        `\nWITH PersonnelId AS (SELECT id FROM personnel WHERE LOWER(email) = '${l.email.toLowerCase()}')
        INSERT INTO bcws_personnel_language ("personnel_id", "language", "level", "level_type")
        SELECT "user_id", "language", "level", "level_type"
        FROM (
            ${selectLanguages.join('\nUNION ALL\n')}
        ) as subquery;\n`);
}
await fs.appendFileSync(sqlPath, `\n\n\n`);


In [633]:
// Net new Tools
for (const t of tools) {
    if (t.tools.length === 0) { continue; }
    if (!pushbackEmailList[t.email?.toLowerCase()]) { continue; }
    const selectTools = t.tools.reduce((acc, tool) => {
        acc.push(`SELECT id AS user_id, ${tool.tool} AS tool, '${tool.proficiencyLevel}'::"tools-proficiency" AS level FROM PersonnelId`);
        return acc;
    }, []);
    await fs.appendFileSync(sqlPath,
        `\nWITH PersonnelId AS (SELECT id FROM personnel WHERE LOWER(email) = '${t.email.toLowerCase()}')
        INSERT INTO bcws_personnel_tools ("personnel_id", "tool_id", "level")
        SELECT "user_id", "tool", "level"
        FROM (
            ${selectTools.join('\nUNION ALL\n')}
        ) as subquery;\n`);
}
await fs.appendFileSync(sqlPath, `\n\n\n`);


In [634]:
// Net new certs
for (const c of certs) {
    if (c.certs.length === 0) { continue; }
    if (!pushbackEmailList[c.email?.toLowerCase()]) { continue; }
    const selectCerts = c.certs.reduce((acc, cert) => {
        acc.push(`SELECT id AS user_id, ${cert.cert} AS cert FROM PersonnelId`);
        return acc;
    }, []);
    await fs.appendFileSync(sqlPath,
        `\nWITH PersonnelId AS (SELECT id FROM personnel WHERE LOWER(email) = '${c.email.toLowerCase()}')
        INSERT INTO bcws_personnel_certifications ("personnel_id", "certification_id")
        SELECT "user_id", "cert"
        FROM (
            ${selectCerts.join('\nUNION ALL\n')}
        ) as subquery;\n`);
}
await fs.appendFileSync(sqlPath, `\n\n\n`);
